In [119]:
import requests
import re
from xml.etree import ElementTree
import numpy as np

In [96]:
base_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/e{}.fcgi'
pubmed_dict = {'db':'pubmed', 'retmode':'json', 'field':'abstract', 'retmax':500,
              'term':'pathways OR mechanism OR mechanisms OR pathway OR underpinnings AND understood OR unclear OR unknown'}

In [97]:
pubmed_json = requests.get(base_url.format('search'), params=pubmed_dict).json()

In [98]:
pubmed_ids = pubmed_json['esearchresult']['idlist']
len(pubmed_ids)

500

In [99]:
#abstract_dict = {'db':'pubmed', 'retmode':'text', # only return mode option is text
#                 'rettype':'abstract',
#              'id':pubmed_ids}
abstract_dict = {'db':'pubmed', 'retmode':'xml', # only return mode option is text
              'id':','.join(pubmed_ids)}
pubmed_xml = requests.get(base_url.format('fetch'), params=abstract_dict)

Pubmed returns an XML. It is structured PubmedArticleList / PubmedArticle / MedlineCitation. MedlineCitation has children: PMID, and Article

Article has children: Journal, ArticleTitle, and Abstract

Abstract has child: AbstractText

In [100]:
pubmed_tree = ElementTree.fromstring(pubmed_xml.text)

In [101]:
abstracts = [ abstract.text for abstract in pubmed_tree.findall('./PubmedArticle/MedlineCitation/Article/Abstract/AbstractText')]
len(abstracts)

1000

In [78]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [125]:
def get_mech_sent(cur_abstract):
    mech_words = {'mechanism', 'mechanisms', 'pathway', 'pathways', 'underpinnings'}
    understood_words = {'understood', 'unknown', 'unclear'}
    cur_sents = sent_tokenize(cur_abstract)
    mech_sent = [sent for sent in cur_sents if mech_words.intersection(set(  word_tokenize(sent) ))]
    understood_sent = [sent for sent in mech_sent if understood_words.intersection(set(  word_tokenize(sent) ))]
    understood_sent = [x for x in understood_sent if x]
    return understood_sent

In [126]:
good_sent = list(filter(len, map(get_mech_sent, abstracts) ))
good_sent = np.array([x[0] for x in good_sent])

In [127]:
good_sent

array([ 'The underlying mechanisms are not fully understood but include impaired mucociliary clearance and structural remodeling of the airways.',
       'Nonetheless, the mechanisms of polarization and strain coupling between grains of different nature in the composites remain unclear.',
       'Previous studies have demonstrated that VEGF affects bone repair and regeneration; however, the cellular mechanisms by which it works are not fully understood.',
       'The underlying mechanisms of apoptosis in HSCs, however, are unknown under consideration of HBV infection.',
       'However, the mechanism of non-leaking CME has been unclear.',
       'The mechanism of how HSP90 inhibition leads to clearance of misfolded proteins is not fully understood.',
       'Context Ginseng is a widely used herbal medicine in China but its mechanism of action remains unclear.',
       'Because the brain mechanisms that underlie these two attention processes are poorly understood, we recorded local fiel